# Script to try setting up Google Earth Engine Authentication and load some sample data

In [5]:
import ee
ee.Initialize(project='ee-ninaimmenroth')


# Load NDVI per tree
creates a table per tree with monthly aggregate features about ndvi (mean, median, std, #observations)

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Load GeoJSON file of merged data set
# df_trees = gpd.read_file('data/berlin_strassenbaeume.geojson')
df_trees = gpd.read_file(
    "data/berlin_trees_with_watering.gpkg",
    layer="trees_watering"
)

print(df_trees.head())

                               id_x              gisid              pitid  \
0  strassenbaeume.00008100_000bbafb  00008100_000bbafb  00008100:000bbafb   
1  strassenbaeume.00008100_000bbafd  00008100_000bbafd  00008100:000bbafd   
2  strassenbaeume.00008100_000bbafe  00008100_000bbafe  00008100:000bbafe   
3  strassenbaeume.00008100_000bbaff  00008100_000bbaff  00008100:000bbaff   
4  strassenbaeume.00008100_000bbb00  00008100_000bbb00  00008100:000bbb00   

  standortnr kennzeich              namenr                art_dtsch  \
0         93     01414  Fritz-Reuter-Allee      Pyramiden-Hainbuche   
1         91     01414  Fritz-Reuter-Allee  Berg-Ahorn, Weiss-Ahorn   
2         90     01414  Fritz-Reuter-Allee  Berg-Ahorn, Weiss-Ahorn   
3         89     01414  Fritz-Reuter-Allee  Berg-Ahorn, Weiss-Ahorn   
4         88     01414  Fritz-Reuter-Allee  Berg-Ahorn, Weiss-Ahorn   

                         art_bot gattung_deutsch   gattung  ... stammumfg  \
0  Carpinus betulus 'Fastigiata' 

In [3]:
df_ch_wilm = df_trees[(df_trees['bezirk'] == 'Charlottenburg-Wilmersdorf') & (df_trees['has_watering'] == True)]
print(df_ch_wilm.head())
df_ch_wilm.shape

                                   id_x              gisid              pitid  \
12784  strassenbaeume.00008100_000c09d5  00008100_000c09d5  00008100:000c09d5   
12785  strassenbaeume.00008100_000c09d7  00008100_000c09d7  00008100:000c09d7   
12791  strassenbaeume.00008100_000c09e5  00008100_000c09e5  00008100:000c09e5   
12792  strassenbaeume.00008100_000c09e9  00008100_000c09e9  00008100:000c09e9   
12793  strassenbaeume.00008100_000c09ea  00008100_000c09ea  00008100:000c09ea   

      standortnr kennzeich           namenr       art_dtsch  \
12784         49     01380  Friedbergstraße  Echter Rotdorn   
12785         48     01380  Friedbergstraße  Echter Rotdorn   
12791         34     01380  Friedbergstraße  Echter Rotdorn   
12792         30     01380  Friedbergstraße  Echter Rotdorn   
12793         29     01380  Friedbergstraße  Echter Rotdorn   

                                    art_bot gattung_deutsch    gattung  ...  \
12784  Crataegus laevigata 'Paul´s Scarlet'        WEIß

(1378, 27)

In [4]:
df_ch_wilm_1 = df_ch_wilm.head(10)

In [9]:
df_ch_wilm.geometry.iloc[0].__geo_interface__

{'type': 'Point', 'coordinates': (13.29205718543995, 52.5036627990978)}

In [ ]:
features = [
    ee.Feature(
        ee.Geometry(g.__geo_interface__),
        {"tree_id": pid}
    )
    for g, pid in zip(
        df_ch_wilm_1.geometry,
        df_ch_wilm_1.pitid
    )
]

trees_fc = ee.FeatureCollection(features)

In [14]:
trees_fc_buffered = trees_fc.map(lambda f: f.buffer(10))

In [15]:
def add_ndvi(img):
    ndvi = img.normalizedDifference(["B8", "B4"]).rename("NDVI")
    return img.addBands(ndvi)

s2 = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterBounds(trees_fc_buffered)
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
    .map(add_ndvi)
)

In [16]:
def monthly_ndvi_stats(year, month):
    start = ee.Date.fromYMD(year, month, 1)
    end = start.advance(1, "month")

    monthly = s2.filterDate(start, end).select("NDVI")

    reducer = (
        ee.Reducer.mean()
        .combine(ee.Reducer.median(), True)
        .combine(ee.Reducer.stdDev(), True)
        .combine(ee.Reducer.count(), True)
    )

    stats_img = monthly.reduce(reducer)

    return stats_img.sampleRegions(
        collection=trees_fc_buffered,
        scale=10,
        geometries=False
    ).map(lambda f: f.set({"year": year, "month": month}))

In [27]:
def monthly_ndvi_quality(year, month):
    start = ee.Date.fromYMD(year, month, 1)
    end = start.advance(1, "month")

    monthly = s2.filterDate(start, end)

    # Select best NDVI pixel per location
    best_img = monthly.qualityMosaic("NDVI").select("NDVI")

    return best_img.reduceRegions(
        collection=trees_fc_buffered,
        reducer=ee.Reducer.mean(),
        scale=10
    ).map(lambda f: f.set({
        "year": year,
        "month": month
    }))


In [28]:
months = list(range(3, 10))  # March–September
year = 2022

fc_list = [
    monthly_ndvi_quality(year, m)
    for m in months
]

ndvi_fc = ee.FeatureCollection(fc_list).flatten()


In [29]:
ndvi_fc_nogeom = ndvi_fc.map(
    lambda f: ee.Feature(
        None,
        f.toDictionary([
            "tree_id",
            "year",
            "month",
            "NDVI"
        ])
    )
)


In [30]:
ndvi_fc_nogeom.limit(5).getInfo()


{'type': 'FeatureCollection',
 'columns': {},
 'features': [{'type': 'Feature',
   'geometry': None,
   'id': '0_0',
   'properties': {'month': 3, 'tree_id': '00008100:000c09d5', 'year': 2022}},
  {'type': 'Feature',
   'geometry': None,
   'id': '0_1',
   'properties': {'month': 3, 'tree_id': '00008100:000c09d7', 'year': 2022}},
  {'type': 'Feature',
   'geometry': None,
   'id': '0_2',
   'properties': {'month': 3, 'tree_id': '00008100:000c09e5', 'year': 2022}},
  {'type': 'Feature',
   'geometry': None,
   'id': '0_3',
   'properties': {'month': 3, 'tree_id': '00008100:000c09e9', 'year': 2022}},
  {'type': 'Feature',
   'geometry': None,
   'id': '0_4',
   'properties': {'month': 3, 'tree_id': '00008100:000c09ea', 'year': 2022}}]}

In [22]:
task = ee.batch.Export.table.toAsset(
    collection=ndvi_fc_nogeom,
    description='ndvi_ee_asset',
    assetId='projects/ee-ninaimmenroth/berlin_tree_ndvi'
)
task.start()


EEException: Request payload size exceeds the limit: 10485760 bytes.

In [15]:
task = ee.batch.Export.table.toDrive(
    collection=ndvi_fc,
    description="berlin_tree_monthly_ndvi_2022_2025",
    fileFormat="CSV"
)

task.start()

EEException: Request payload size exceeds the limit: 10485760 bytes.

## Load a small sample
Pick a location (e.g. Alexanderplatz) and load a small sample for a certain timeframe.

In [ ]:
roi = ee.Geometry.Point([13.4125, 52.5218]).buffer(500)

s2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
      .filterBounds(roi)
      .filterDate("2023-06-01", "2023-06-30")
      .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10)))

def add_ndvi(img):
    ndvi = img.normalizedDifference(["B8", "B4"]).rename("NDVI")
    return img.addBands(ndvi)

s2_ndvi = s2.map(add_ndvi)

# This extracts random sample points from the first image

img = s2_ndvi.first()

samples = img.select("NDVI").sample(
    region=roi,
    scale=10,
    numPixels=20,   # sample size
    seed=42
)


## Download as a pandas dataframe

In [ ]:
import pandas as pd

# Convert Earth Engine FeatureCollection → client-side dict
data = samples.getInfo()

# Convert to pandas DataFrame
df = pd.DataFrame([f["properties"] for f in data["features"]])
df.head()


,NDVI
0,0.276256
1,0.091916
2,0.117857
3,0.091879
4,0.549828


In [ ]:
df.to_csv("ndvi_sample.csv", index=False)


## Download NDVI png of Berlin

In [ ]:
# geometry
berlin = ee.Geometry.Point([13.4050, 52.5200]).buffer(5000)

# select image
s2 = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterBounds(berlin)
    .filterDate("2023-06-01", "2023-09-01")
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 30))
    .sort("CLOUDY_PIXEL_PERCENTAGE")
    .first()
)

# assert bands
print("bands:", s2.bandNames().getInfo())

# ndvi
ndvi = s2.normalizedDifference(["B8", "B4"]).rename("NDVI")

# style
ndvi_vis = ndvi.visualize(
    min=0.0,
    max=1.0,
    palette=['blue','white','green']
)

# thumbnail
thumb_url = ndvi_vis.getThumbURL({
    "region": berlin,
    "dimensions": 800,
    "format": "png"
})

print("Download NDVI:", thumb_url)

bands: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60', 'MSK_CLASSI_OPAQUE', 'MSK_CLASSI_CIRRUS', 'MSK_CLASSI_SNOW_ICE']
Download NDVI: https://earthengine.googleapis.com/v1/projects/ee-ninaimmenroth/thumbnails/3c337696cfd68754637298942be39a4f-af80adf688908d8f562067527268478f:getPixels
